In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import StratifiedShuffleSplit
import xgboost as xgb
import datetime
from pandas.api.types import CategoricalDtype

In [2]:
train = pd.read_csv('/Users/young/dataset_local/walmart-recruiting-trip-type-classification/train.csv',dtype=({'Upc':object,'FinelineNumber':object,'ScanCount':np.int16,'TripType':np.int16,'VisitNumber':np.int32}))
test = pd.read_csv('/Users/young/dataset_local/walmart-recruiting-trip-type-classification/test.csv',dtype=({'Upc':object,'FinelineNumber':object,'ScanCount':np.int16,'TripType':np.int16,'VisitNumber':np.int32}))

In [3]:
test['TripType'] = 0
test['TripType']

0         0
1         0
2         0
3         0
4         0
         ..
653641    0
653642    0
653643    0
653644    0
653645    0
Name: TripType, Length: 653646, dtype: int64

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647054 entries, 0 to 647053
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   TripType               647054 non-null  int16 
 1   VisitNumber            647054 non-null  int32 
 2   Weekday                647054 non-null  object
 3   Upc                    642925 non-null  object
 4   ScanCount              647054 non-null  int16 
 5   DepartmentDescription  645693 non-null  object
 6   FinelineNumber         642925 non-null  object
dtypes: int16(2), int32(1), object(4)
memory usage: 24.7+ MB


In [5]:
tr_VisitNumber = list(train.VisitNumber.unique())
te_VisitNumber = list(test.VisitNumber.unique())

In [6]:
train.Upc.fillna('0',inplace=True)
train.DepartmentDescription.fillna('NA',inplace=True)
train.FinelineNumber.fillna('-1',inplace=True)
test.Upc.fillna('0',inplace=True)
test.DepartmentDescription.fillna('NA',inplace=True)
test.FinelineNumber.fillna('-1',inplace=True)


In [7]:
train.Upc = train.Upc.astype(np.int64)
train.FinelineNumber = train.FinelineNumber.astype(np.int16)
train.ScanCount = train.ScanCount.astype(np.int32)
test.Upc = test.Upc.astype(np.int64)
test.FinelineNumber = test.FinelineNumber.astype(np.int16)
test.ScanCount = test.ScanCount.astype(np.int32)

In [8]:
train.sort_values(by='VisitNumber',inplace=True)
train[train['DepartmentDescription']=='PHARMACY RX']

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
1155,44,496,Friday,0,1,PHARMACY RX,-1
1216,5,521,Friday,0,1,PHARMACY RX,-1
1373,5,585,Friday,0,1,PHARMACY RX,-1
1456,5,619,Friday,0,1,PHARMACY RX,-1
1457,5,619,Friday,0,1,PHARMACY RX,-1
...,...,...,...,...,...,...,...
636715,5,188839,Sunday,0,1,PHARMACY RX,-1
636716,5,188839,Sunday,0,1,PHARMACY RX,-1
636717,5,188839,Sunday,0,1,PHARMACY RX,-1
636847,5,188896,Sunday,0,1,PHARMACY RX,-1


In [23]:
train_visitnumber_triptype = train.groupby(['VisitNumber']).agg({'TripType': 'first'}).reset_index()
train_visitnumber_triptype.head()

,VisitNumber,TripType
0,5,999
1,7,30
2,8,26
3,9,8
4,10,8


In [11]:
# The train and test dataframes are conbined together, so the features are dealt together
train_test = pd.concat([train,test], ignore_index=True).sort_values('VisitNumber')
train_test.TripType = train_test.TripType.astype(np.int16)
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300700 entries, 647054 to 1300699
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   TripType               1300700 non-null  int16 
 1   VisitNumber            1300700 non-null  int32 
 2   Weekday                1300700 non-null  object
 3   Upc                    1300700 non-null  int64 
 4   ScanCount              1300700 non-null  int32 
 5   DepartmentDescription  1300700 non-null  object
 6   FinelineNumber         1300700 non-null  int16 
dtypes: int16(2), int32(2), int64(1), object(2)
memory usage: 54.6+ MB


In [17]:
# Calculate the total number of items purchased per each visit
sCountPerVisit = train_test.groupby(['VisitNumber']).agg({'ScanCount': 'sum'})\
.rename(columns={'ScanCount': 'SCountPerVisit'}).reset_index()
sCountPerVisit.head()

,VisitNumber,SCountPerVisit
0,1,4
1,2,4
2,3,0
3,4,1
4,5,-1


In [18]:
# Calculate the total number of items per visit per Upc
sCountPerVisitPerUpc = train_test.groupby(['VisitNumber', 'Upc']).agg({'ScanCount': 'sum'})\
.rename(columns={'ScanCount': 'SCountPerVisitPerUpc'}).reset_index()
sCountPerVisitPerUpc.head()

,VisitNumber,Upc,SCountPerVisitPerUpc
0,1,1707710732,1
1,1,72503389714,1
2,1,88491211470,1
3,1,89470001026,1
4,2,2840015224,1


In [19]:
# Convert weekday to number
wdict = {'Monday':1,
        'Tuesday':2,
        'Wednesday':3,
        'Thursday':4,
        'Friday':5,
        'Saturday':6,
        'Sunday':7}

train_test['wd'] = train_test.Weekday.apply(lambda x: wdict[x])

In [21]:
# netout_visits include visits which the TotalItems <= 0
netout_visits = list(sCountPerVisit[sCountPerVisit.SCountPerVisit <= 0]['VisitNumber'])
# In the train set, these visits types are almost always 999
train_netout_visits = train_visitnumber_triptype[train_visitnumber_triptype.VisitNumber.isin(netout_visits)]
print(np.count_nonzero(train_netout_visits.TripType == 999) / train_netout_visits.shape[0] *100,'% of these visits are of TripType 999')

99.93546305259761 % of these visits are of TripType 999


In [26]:
# someout_visits include visits where the total ScanCount of some Upc items is less than 0
someout_visits = list(sCountPerVisitPerUpc[sCountPerVisitPerUpc.SCountPerVisitPerUpc < 0]['VisitNumber'])
# In the train set, these visits types are almost always 999
train_someout_visits = train_visitnumber_triptype[train_visitnumber_triptype.VisitNumber.isin(someout_visits)]
print(np.count_nonzero(train_someout_visits.TripType == 999) / train_someout_visits.shape[0] *100,'% of these visits are of TripType 999')

99.80995819080198 % of these visits are of TripType 999


In [27]:
# Therefore it is safe to assign 999 to these visits, so they are removed for now.
train_test = train_test.query('VisitNumber != @netout_visits & VisitNumber != @someout_visits')
train_test.shape

(1274023, 8)

In [28]:
# Add columns 'Pos' and 'Neg'.
# They are correlated but are useful to mark those return records, before aggregating ScanCount
train_test['Pos'] = (train_test.ScanCount > 0).astype(np.int16)
train_test['Neg'] = (train_test.ScanCount < 0).astype(np.int16)

/Users/young/opt/anaconda3/envs/test1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/young/opt/anaconda3/envs/test1/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
# Aggregate ScanCount
train_test = train_test.groupby(['VisitNumber',
 'Upc',
 'DepartmentDescription',
 'Weekday',
 'FinelineNumber',
 'TripType'], as_index=False).sum().sort_values('VisitNumber')
# Add column Return, which is the sign of ScanCount
train_test['Return'] = train_test.ScanCount.map(lambda x: np.sign(x)).astype(np.int16)


,VisitNumber,Upc,DepartmentDescription,Weekday,FinelineNumber,TripType,ScanCount,wd,Pos,Neg,Return
1246433,191348,88181390024,BATH AND SHOWER,Sunday,1099,0,1,7,1,0,1
1246429,191348,7871535983,MENS WEAR,Sunday,4923,0,1,7,1,0,1
1246428,191348,4282557050,MENS WEAR,Sunday,8220,0,1,7,1,0,1
1246430,191348,66572105763,BATH AND SHOWER,Sunday,1505,0,1,7,1,0,1
1246434,191348,88743955571,MENS WEAR,Sunday,1857,0,1,7,1,0,1


In [34]:
# Add time of the day as a fraction of one day, 0 is the first visit of the day, 1 means the last visit of the day
# 'first' is the first VisitNumber of the day
#visitnumber이 순서대로 찍히는 점을 이용해서, 날짜별 마지막 visitnumber를 활용해서 방문 시간 계산
train_test['first'] = (train_test
               .groupby((train_test.Weekday != train_test.Weekday.shift()).cumsum())
               .VisitNumber
               .transform('first'))
# 'last' is the last VisitNumber of the day
train_test['last'] =  (train_test
               .groupby((train_test.Weekday != train_test.Weekday.shift()).cumsum())
               .VisitNumber
               .transform('last'))
train_test['time_of_day'] = (train_test['VisitNumber'] - train_test['first'] + 1) / (train_test['last'] - train_test['first'] + 1)

In [44]:
train_test.head()

,VisitNumber,Upc,DepartmentDescription,Weekday,FinelineNumber,TripType,ScanCount,wd,Pos,Neg,Return,first,last,time_of_day,day_counter
0,1,1707710732,DAIRY,Friday,1526,0,1,5,1,0,1,1,6213,0.000161,1
1,1,72503389714,SHOES,Friday,3002,0,1,5,1,0,1,1,6213,0.000161,1
2,1,88491211470,GROCERY DRY GOODS,Friday,3555,0,1,5,1,0,1,1,6213,0.000161,1
3,1,89470001026,DAIRY,Friday,1431,0,1,5,1,0,1,1,6213,0.000161,1
4,2,2840015224,DSD GROCERY,Friday,4408,0,1,5,1,0,1,1,6213,0.000322,1


In [45]:
# Create a column 'day_counter'. There are 31 days in total
#순서대로 쌓인 점을 이용해서, 행별로 Weekday가 이전 행과 달라지지 않았따면 같은날로, 이전행과 달라졌다면 +1일로 계산.
train_test['day_counter'] = (train_test.Weekday != train_test.Weekday.shift()).cumsum()